In [1]:
import glymur
import matplotlib.pyplot as plt
import julian
import numpy as np
import pandas as pd
import os
from datetime import datetime, timedelta

def parse_metadata(file_path):
    metadata = {}
    enc = 'utf-8'
    try:
        with open(file_path, 'r', encoding=enc) as file:
            for line in file:
                if "=" in line:
                    key, value = line.split("=", 1)
                    key = key.strip()
                    value = value.strip().strip('"')
                    metadata[key] = value
    except UnicodeDecodeError:
        print(f"Error decoding the file {file_path} with encoding {enc}")
    return metadata

def extract_jp2_data(jp2_file_path):
    jp2 = glymur.Jp2k(jp2_file_path)
    image_data = jp2[:]
    return image_data

def convert_dn_to_val(dn, scaling_factor, offset, missing_constant):
    val = np.full(dn.shape, np.nan)
    mask = (dn != missing_constant)
    val[mask] = (dn[mask] * scaling_factor) + offset
    return val

def clean_metadata_value(value):
    try:
        cleaned_value = ''.join(filter(lambda x: x.isdigit() or x in ['.', '-'], value))
        return float(cleaned_value)
    except ValueError:
        print(f"Error converting metadata value to float: {value}")
        return None

def generate_coordinates(image_shape, metadata):
    lines, samples = image_shape
    line_projection_offset = clean_metadata_value(metadata['LINE_PROJECTION_OFFSET'])
    sample_projection_offset = clean_metadata_value(metadata['SAMPLE_PROJECTION_OFFSET'])
    map_scale = clean_metadata_value(metadata['MAP_SCALE'])
    center_lat = clean_metadata_value(metadata['CENTER_LATITUDE'])
    center_lon = clean_metadata_value(metadata['CENTER_LONGITUDE'])
    map_resolution = clean_metadata_value(metadata['MAP_RESOLUTION'])

    # Assuming MAP_RESOLUTION is in degrees per pixel
    deg_per_pixel = 1.0 / map_resolution

    lats = center_lat - ((np.arange(lines) - line_projection_offset) * deg_per_pixel)
    lons = center_lon + ((np.arange(samples) - sample_projection_offset) * deg_per_pixel)

    # Ensure lons and lats fall within specified min/max values
    lats = np.clip(lats, clean_metadata_value(metadata['MINIMUM_LATITUDE']), clean_metadata_value(metadata['MAXIMUM_LATITUDE']))
    lons = np.mod(lons, 360.0)

    return np.meshgrid(lons, lats)

def save_to_csv(lons, lats, julian_dates, output_csv_path):
    data = {
        'Longitude': lons.flatten(),
        'Latitude': lats.flatten(),
        'Julian Date': julian_dates.flatten(),
    }
    
    df = pd.DataFrame(data)
    df.to_csv(output_csv_path, index=False)
    print(f"Data saved to {output_csv_path}")

def process_image(metadata, jp2_file_path, output_csv_path, data_type, save_data=True):

    accepted_data_types = ['date', 'temp']
    if data_type not in accepted_data_types:
        raise ValueError(f"Invalid data type '{data_type}'. Accepted values are: {accepted_data_types}")
    
    image_data = extract_jp2_data(jp2_file_path)

    scaling_factor = clean_metadata_value(metadata.get('SCALING_FACTOR', 1))  # Default to 1 if not found
    offset = clean_metadata_value(metadata.get('OFFSET', 0))  # Default to 0 if not found
    missing_constant = (metadata.get('IMAGE_MISSING_CONSTANT', -32768))

    dn_value = image_data[0, 0]

    lons, lats = generate_coordinates(image_data.shape, metadata)

    if data_type == 'date':

        output_vals = convert_dn_to_val(image_data, scaling_factor, offset, missing_constant)
        julian_date = convert_dn_to_val(np.array([dn_value]), scaling_factor, offset, missing_constant)[0]

        if ~np.isnan(julian_date):
            calendar_date = julian.from_jd(julian_date, fmt='jd')
            print(f"Digital Number (DN): {dn_value} -> Julian Date: {julian_date} -> Gregorian Date: {calendar_date}\n")

    elif data_type == 'temp':
        output_vals = convert_dn_to_val(image_data, scaling_factor, offset, missing_constant)

    if save_data:
        save_to_csv(lons, lats, output_vals, output_csv_path)

    df = pd.DataFrame({
        'Longitude': lons.flatten(),
        'Latitude': lats.flatten(),
        data_type: output_vals.flatten(),
    })

    return df


In [2]:
    # Optimize the data types of the dataframes
    def optimize_df(df):
        for col in df.select_dtypes(include=['float64']).columns:
            df[col] = pd.to_numeric(df[col], downcast='float')
        for col in df.select_dtypes(include=['int64']).columns:
            df[col] = pd.to_numeric(df[col], downcast='integer')
        return df

In [3]:
date_metadata_file_path = '/home/as5023/ACSE/irp-as5023/data/Diviner-temp/dgdr_jd_avg_poln_20160111d_240_jp2.lbl'
date_metadata = parse_metadata(date_metadata_file_path)
date_jp2_file_path = date_metadata_file_path.replace('.lbl', '.jp2')

temp_metadata_file_path = '/home/as5023/ACSE/irp-as5023/data/Diviner-temp/dgdr_tbol_avg_poln_20160111d_240_jp2.lbl'
temp_metadata = parse_metadata(temp_metadata_file_path)
temp_jp2_file_path = temp_metadata_file_path.replace('.lbl', '.jp2')

output_csv_path = 'diviner_data.csv'

if os.path.exists(date_jp2_file_path) and os.path.exists(temp_jp2_file_path):
    date_df = process_image(date_metadata, date_jp2_file_path, output_csv_path, 'date', False)
    temp_df = process_image(temp_metadata, temp_jp2_file_path, output_csv_path, 'temp', False)

    date_df = optimize_df(date_df)
    temp_df = optimize_df(temp_df)
    date_df['date'] = pd.to_numeric(date_df['date'], errors='coerce')
    temp_df['temp'] = pd.to_numeric(temp_df['temp'], errors='coerce')
    print(f"Number of Nans in Julian Dates: {np.isnan(date_df["date"]).sum()} out of {np.prod(date_df["date"].shape)} ({(np.isnan(date_df["date"]).sum()/np.prod(date_df["date"].shape)*100):.2f}%)")
    print(f"Number of Nans in Temperatures: {np.isnan(temp_df["temp"]).sum()} out of {np.prod(temp_df["temp"].shape)} ({(np.isnan(temp_df["temp"]).sum()/np.prod(temp_df["temp"].shape)*100):.2f}%)\n")


    # Check if coordinates align
    if not (date_df[['Longitude', 'Latitude']].equals(temp_df[['Longitude', 'Latitude']])):
        raise ValueError("The coordinates in date_df and temp_df do not align")
    
    # #If they align, drop the duplicate columns and concat
    temp_df = temp_df.drop(columns=['Longitude', 'Latitude'])
    combined_df = pd.concat([date_df, temp_df], axis=1)

    # # # Print the combined dataframe's head and other information
    print(combined_df.head(), '\n')
    print(f'Min/max longitude: {combined_df["Longitude"].min()}, {combined_df["Longitude"].max()}')
    print(f'Min/max latitude: {combined_df["Latitude"].min()}, {combined_df["Latitude"].max()}')

    min_jd = np.nanmin(combined_df["date"])
    max_jd = np.nanmax(combined_df["date"])


    if not np.isnan(min_jd) and not np.isnan(max_jd):
        print(f'Min/max Gregorian date: {julian.from_jd(min_jd, fmt="jd")}, {julian.from_jd(max_jd, fmt="jd")}\n')

else:
    print(f"JP2 file not found at {jp2_file_path}")

Number of Nans in Julian Dates: 1870572 out of 14531344 (12.87%)
Number of Nans in Temperatures: 2550840 out of 14531344 (17.55%)

    Longitude  Latitude  date  temp
0  344.918518      90.0   NaN   NaN
1  344.926422      90.0   NaN   NaN
2  344.934357      90.0   NaN   NaN
3  344.942261      90.0   NaN   NaN
4  344.950165      90.0   NaN   NaN 

Min/max longitude: 0.00395735539495945, 359.99603271484375
Min/max latitude: 75.0, 90.0
Min/max Gregorian date: 2016-01-12 18:00:00, 2016-02-10 00:00:00

